In [3]:
import re
import urllib2
from bs4 import BeautifulSoup

In [4]:
import os
import csv

In [5]:
from time import sleep

In [6]:
start_url_base = "http://www.gutenberg.org/wiki/Science_Fiction_(Bookshelf)"

In [7]:
base_url = "https://www.gutenberg.org"

In [8]:
def make_soup(url):
    hdr = {"User-Agent": "Mozilla/5.0"}
    return BeautifulSoup(urllib2.urlopen(urllib2.Request(url,headers=hdr)).read(),'lxml')

In [9]:
def get_book_links(url):
    soup = make_soup(url)
    atagbooks = soup.find_all("a", attrs = {"title": re.compile(r'^ebook:')})
    book_links = []
    for element in atagbooks:
        book_link = "http:" + element["href"]
        book_links.append(book_link)
    return book_links        

In [11]:
def get_book_information(book_url):
    soup = make_soup(book_url + '#bibrec')
    
    if soup.find('th', string='EBook-No.') is not None:
        foundth5 = soup.find('th', string='EBook-No.')
        bookid = foundth5.next_sibling.next_sibling.string
    else:
        bookid = "NA"
        
    nesteddict={}
    nesteddict[bookid]={}
    
    patterns = [r"Author", r"Title", r"Illustrator", r"Language", r"^LoC", r"Subject", 
                r"Category", r"^Release", r"^Copyright", r"Downloads", r"Price"]
    
    for i in patterns:
        infolist = soup.find_all('th', string=re.compile(i))
        if len(infolist) == 1:
            if infolist[0].next_sibling.next_sibling.find("a") is not None:
                info = infolist[0].next_sibling.next_sibling.a.get_text().strip()
            else:
                info = infolist[0].next_sibling.next_sibling.get_text().strip()
        elif len(infolist) == 0:
            info = "NA"
        else:
            infol = []
            for element in infolist:
                if element.next_sibling.next_sibling.find("a") is not None:
                    infom = element.next_sibling.next_sibling.a.string.strip()
                else:
                    infom = infolist[0].next_sibling.next_sibling.get_text().strip()
                infol.append(infom)
            info = '/'.join(infol)
        if i[0] == "^":
            i = i[1:]
        infodict = {i: info}
        nesteddict[bookid].update(infodict)
        
    return nesteddict

In [ ]:
knowlen = get_book_links(start_url_base)
len(knowlen)

1371

In [ ]:
bookmeta = {}

for booklink in get_book_links(start_url_base):
    print "Processing %s" %(booklink) 
    try:
        bookinfo = get_book_information(booklink)
        bookmeta.update(bookinfo)
    except:
        print "Failed to process %s" %(booklink)
        pass
    sleep(0.01)

os.chdir("/Users/tuh48546/Documents/Sci Fi/") # change directory accordingly 
with open("meta1025_scifi_bookshelf.csv", "w") as toWrite: # change filename accordingly
    writer = csv.writer(toWrite, delimiter=",")
    writer.writerow(["Book_ID", "Author", "Title", "Illustrator", "Language", "Category", "Release Date", 
                    "Copyright", "Downloads", "Price", "LoC"])
    for a in bookmeta.keys():
        writer.writerow([a.encode('utf-8'), 
                         bookmeta[a]["Author"].encode('utf-8'), 
                         bookmeta[a]["Title"].encode('utf-8'),
                         bookmeta[a]["Illustrator"].encode('utf-8'),
                         bookmeta[a]["Language"].encode('utf-8'),
                         bookmeta[a]["Category"].encode('utf-8'),
                         bookmeta[a]["Release"].encode('utf-8'),
                         bookmeta[a]["Copyright"].encode('utf-8'),
                         bookmeta[a]["Downloads"].encode('utf-8'),
                         bookmeta[a]["Price"].encode('utf-8'),
                         bookmeta[a]["LoC"].encode('utf-8')])

Processing http://www.gutenberg.org/ebooks/19445
Processing http://www.gutenberg.org/ebooks/10002
Processing http://www.gutenberg.org/ebooks/41481
Processing http://www.gutenberg.org/ebooks/28617
Processing http://www.gutenberg.org/ebooks/29607
Processing http://www.gutenberg.org/ebooks/29390
Processing http://www.gutenberg.org/ebooks/29809
Processing http://www.gutenberg.org/ebooks/29848
Processing http://www.gutenberg.org/ebooks/29198
Processing http://www.gutenberg.org/ebooks/29768
Processing http://www.gutenberg.org/ebooks/29255
Processing http://www.gutenberg.org/ebooks/29882
Processing http://www.gutenberg.org/ebooks/29919
Processing http://www.gutenberg.org/ebooks/30691
Processing http://www.gutenberg.org/ebooks/30177
Processing http://www.gutenberg.org/ebooks/30124
Processing http://www.gutenberg.org/ebooks/30166
Processing http://www.gutenberg.org/ebooks/30452
Processing http://www.gutenberg.org/ebooks/30532
Processing http://www.gutenberg.org/ebooks/31893
Processing http://ww

Processing http://www.gutenberg.org/ebooks/23790
Processing http://www.gutenberg.org/ebooks/20707
Processing http://www.gutenberg.org/ebooks/20988
Processing http://www.gutenberg.org/ebooks/20154
Processing http://www.gutenberg.org/ebooks/13083
Processing http://www.gutenberg.org/ebooks/23146
Processing http://www.gutenberg.org/ebooks/51519
Processing http://www.gutenberg.org/ebooks/27968
Processing http://www.gutenberg.org/ebooks/26855
Processing http://www.gutenberg.org/ebooks/17958
Processing http://www.gutenberg.org/ebooks/32207
Processing http://www.gutenberg.org/ebooks/24977
Processing http://www.gutenberg.org/ebooks/29632
Processing http://www.gutenberg.org/ebooks/40970
Processing http://www.gutenberg.org/ebooks/31922
Processing http://www.gutenberg.org/ebooks/29625
Processing http://www.gutenberg.org/ebooks/18668
Processing http://www.gutenberg.org/ebooks/26968
Processing http://www.gutenberg.org/ebooks/50928
Processing http://www.gutenberg.org/ebooks/32181
Processing http://ww

In [ ]:
# Output note 10/25/2017: 15 failures. Probably due to an additional row called "LoC No." in these 15 books.
# Should consider modify the get_book_information() function and try another scrape later.